In [106]:
# Import section
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers import Normalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import RobustScaler
import pandas as pd
import numpy as np 
import math
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import datetime
from datetime import date, timedelta
from clean import clean_all, get_data, get_bairros_data
from preproc import get_format, get_popfile, clean_pop_data, extract_ts
idx = pd.IndexSlice

In [100]:
# Data loading
data1, data2, data3, data4 = get_data()
data = clean_all(data1, data2, data3, data4, get_bairros_data())
preprocessed_data, preprocessed_data_1000 = get_format(data)
pop_clean = clean_pop_data()


KeyboardInterrupt



In [157]:
preprocessed_data_1000

nb_crimes                                                   \
AR          Anchieta     Bangu Barra da Tijuca  Botafogo Campo Grande   
Date                                                                    
2008-12-31  0.025522  0.009079        0.011007  0.000000     0.010674   
2009-01-01  0.063703  0.047398        0.052289  0.057072     0.066402   
2009-01-02  0.031852  0.015799        0.010458  0.009512     0.045433   
2009-01-03  0.006370  0.040626        0.017430  0.014268     0.027959   
2009-01-04  0.006370  0.029341        0.006972  0.014268     0.033201   
...              ...       ...             ...       ...          ...   
2019-12-27  0.031555  0.017175        0.028888  0.082877     0.016165   
2019-12-28  0.044177  0.034350        0.022698  0.049726     0.019104   
2019-12-29  0.056799  0.015028        0.018571  0.060776     0.010287   
2019-12-30  0.031555  0.030056        0.024761  0.055251     0.011756   
2019-12-31  0.000000  0.000000        0.000000  0.000000     0.001470   

                                                                             \
AR            Centro Cidade de Deus Complexo do Alemao Copacabana Guaratiba   
Date                                                                          
2008-12-31  0.066617       0.000000           0.000000   0.034691  0.050378   
2009-01-01  0.103458       0.198661           0.000000   0.070387  0.083396   
2009-01-02  0.206917       0.000000           0.000000   0.056310  0.015163   
2009-01-03  0.103458       0.198661           0.000000   0.042232  0.037907   
2009-01-04  0.206917       0.099331           0.000000   0.035194  0.015163   
...              ...            ...                ...        ...       ...   
2019-12-27  1.117004       0.000000           0.000000   0.073479  0.007922   
2019-12-28  0.777046       0.000000           0.000000   0.155123  0.011884   
2019-12-29  0.339958       0.062570           0.016988   0.236767  0.003961   
2019-12-30  0.485654       0.031285           0.000000   0.081644  0.011884   
2019-12-31  0.000000       0.000000           0.000000   0.008164  0.000000   

            ...                                                       \
AR          ... Portuaria     Ramos  Realengo Rio Comprido   Rocinha   
Date        ...                                                        
2008-12-31  ...  0.000000  0.006877  0.026449     0.015969  0.000000   
2009-01-01  ...  0.029703  0.062168  0.046459     0.114169  0.014499   
2009-01-02  ...  0.000000  0.034538  0.025811     0.016310  0.000000   
2009-01-03  ...  0.059406  0.041445  0.041297     0.048929  0.014499   
2009-01-04  ...  0.000000  0.034538  0.036135     0.114169  0.000000   
...         ...       ...       ...       ...          ...       ...   
2019-12-27  ...  0.108767  0.036309  0.047438     0.060599  0.000000   
2019-12-28  ...  0.072512  0.036309  0.038813     0.040399  0.000000   
2019-12-29  ...  0.036256  0.029047  0.038813     0.080799  0.000000   
2019-12-30  ...  0.072512  0.021785  0.017250     0.000000  0.011689   
2019-12-31  ...  0.000000  0.000000  0.000000     0.000000  0.000000   

                                                                        
AR         Santa Cruz Santa Teresa Sao Cristovao    Tijuca Vila Isabel  
Date                                                                    
2008-12-31   0.002863     0.057947      0.052674  0.006297    0.000000  
2009-01-01   0.062111     0.000000      0.072225  0.012807    0.053311  
2009-01-02   0.028232     0.192864      0.018056  0.025613    0.038079  
2009-01-03   0.014116     0.096432      0.054169  0.019210    0.015232  
2009-01-04   0.042349     0.000000      0.126394  0.012807    0.007616  
...               ...          ...           ...       ...         ...  
2019-12-27   0.007407     0.033653      0.071630  0.091264    0.050575  
2019-12-28   0.004938     0.067305      0.071630  0.022816    0.057800  
2019-12-29   0.012346     0.067305      0.071630  0.022816    0.02890

In [164]:
# Preprocessing data
def subsample_sequence(data, length): # Return a shorter dataframe with specified 
                                                 # length for a specific barrio
    last_possible = data.shape[0] - length
    random_start = np.random.randint(0, last_possible)
    data_sample = data[random_start: random_start+length]

    return data_sample

def split_subsample_sequence(data, length): # Return a random sequence of specified length

    data_subsample = subsample_sequence(data, length)
    y_sample = data_subsample.iloc[length-31:]
    
    X_sample = data_subsample[0:length-31]
    X_sample = X_sample.values
    return np.array(X_sample), np.array(y_sample)

def get_X_y(data, n_sequences, length): # Return a sepcific number of (X,y) samples of specified length
                                                   # for a specified bairro

    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(data, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

def get_train_test(data,n_sequences,length): # Return train and test data

    len_ = int(0.8*data.shape[0])
    data_train = data[:len_]
    data_test = data[len_:]
    
    test_seq = math.floor(n_sequences/4)
    
    X_train, y_train = get_X_y(data_train, n_sequences, length)
    X_test, y_test = get_X_y(data_test, test_seq, length)
    
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],1)
    #X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)
    
    return X_train, y_train
    # X_test, y_test

In [163]:
 # Loading X_train/test, y_train/test

data = get_train_test(preprocessed_data_1000, 500, 120)
data

ValueError: cannot reshape array of size 1335000 into shape (500,89,1)

In [156]:
# Outside preprocessing 

for AR in AR_list:
    normalizer.adapt(all_data[AR][0])
    normalizer.adapt(all_data[AR][2])

In [145]:
# Creating a model
def init_model(AR):
    metric = metrics.MAPE
    opt = optimizers.RMSprop(learning_rate=0.005)
    
    model = models.Sequential()
    model.add(layers.LSTM(30, return_sequences=True, activation='tanh'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(31, activation='linear'))
    
    model.compile(loss='mse', 
                  optimizer=opt, 
                  metrics=[metric])
    
    return model

# Parameters
# def get_parameters(): 
es = EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)

# Fitting the model
def fit(model, AR):
    hist = model.fit(all_data[AR][0], all_data[AR][1],
            validation_split=0.3,
            epochs=5, 
            batch_size=32,
            callbacks=[es], verbose=0)
    return hist

# # Evaluating the model 
# def evaluate(hist, AR):
#     results = hist.model.evaluate(all_data[AR][2], all_data[AR][3])
#     return results

# # All-in-1
# def modelling(AR):
#     results = evaluate(fit(init_model(AR), AR), AR)
#     return results

In [138]:
model = init_model('Centro')
fitted_model = fit(model, 'Centro')

In [146]:
results = evaluate(fitted_model, 'Centro')
results

16/16 [==============================] - 0s 26ms/step - loss: 0.6859 - mean_absolute_percentage_error: 29.7326


[0.685863733291626, 29.732606887817383]

In [147]:
# Getting the results for each AR

all_results = {}

for AR in AR_list:
    all_results[AR] = modelling(AR)
    
all_results

16/16 [==============================] - 0s 25ms/step - loss: 0.0030 - mean_absolute_percentage_error: 6907.5859


{'Anchieta': [0.0007052274304442108, 43.19216537475586],
 'Bangu': [0.00038059073267504573, 31.299766540527344],
 'Barra da Tijuca': [0.000483615614939481, 49.03586959838867],
 'Botafogo': [0.0023192802909761667, 30.577455520629883],
 'Campo Grande': [5.748889816459268e-05, 31.537446975708008],
 'Centro': [0.7550690770149231, 28.683307647705078],
 'Cidade de Deus': [0.000908440153580159, 19729296.0],
 'Complexo do Alemao': [0.00013193904305808246, 6677155.5],
 'Copacabana': [0.010875527746975422, 55.954063415527344],
 'Guaratiba': [6.901000597281381e-05, 777216.3125],
 'Ilha do Governador': [0.00017346777894999832, 349548.15625],
 'Inhauma': [0.0011083482531830668, 295615.8125],
 'Iraja': [0.0010068053379654884, 28.063663482666016],
 'Jacarepagua': [0.000110312677861657, 27.325624465942383],
 'Jacarezinho': [5.9099231293657795e-05, 3595526.75],
 'Lagoa': [0.6943167448043823, 76.94197082519531],
 'Madureira': [0.0007411403930746019, 25.924497604370117],
 'Mare': [0.0001275737740797922, 

In [152]:
# getting the mean of all the results
mape = [all_results[AR][1] for AR in all_results]
mape_overall = np.mean(mape)
mape_overall

1822820.9762325287

In [113]:
# Plotting the results
import matplotlib.pyplot as plt
plt.plot(history.history['mean_absolute_percentage_error'])
plt.plot(history.history['val_mean_absolute_percentage_error'])

NameError: name 'history' is not defined

In [ ]:
# Printing the results

res = model.evaluate(X_test, y_test, verbose=0)
print(f'MAPE on the test set : {res[1]:.0f} %')b

In [155]:
all_data['Centro'][2].shape

(500, 169, 1)

In [ ]:
# 